In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pymc3 as pm
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

In [ ]:
df = pd.read_csv("retail.csv")

In [ ]:
df = df[["InvoiceNo", "StockCode"]]
df.dropna(inplace=True)

In [ ]:
basket = df.groupby(["InvoiceNo"])["StockCode"].apply(list).reset_index()

In [ ]:
te = TransactionEncoder()
te_ary = te.fit(basket["StockCode"]).transform(basket["StockCode"])
df_basket = pd.DataFrame(te_ary, columns=te.columns_)

In [ ]:
print(df_basket.head())

In [ ]:
frequent_itemsets = apriori(df_basket, min_support=0.02, use_colnames=True)

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.3)

with pm.Model() as model:
    confidence_prior = pm.Beta("confidence_prior", alpha=2, beta=2, shape=len(rules))

    observed_confidence = pm.Normal("observed_confidence", mu=confidence_prior, sigma=0.1, observed=rules["confidence"])

    trace = pm.sample(1000, return_inferencedata=False, cores=1)

In [ ]:
rules["bayesian_conf_low"] = np.percentile(trace["confidence_prior"], 2.5, axis=0)
rules["bayesian_conf_high"] = np.percentile(trace["confidence_prior"], 97.5, axis=0)

print(rules[["antecedents", "consequents", "confidence", "bayesian_conf_low", "bayesian_conf_high"]])

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(trace["confidence_prior"], bins=30, kde=True)
plt.axvline(np.mean(trace["confidence_prior"]), color="red", linestyle="dashed", label="Mean")
plt.xlabel("Confidence")
plt.ylabel("Density")
plt.title("Bayesian Confidence Distribution")
plt.legend()
plt.show()